In [10]:
from models import LatentClassifier, VAE_CELL_CNN

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# mnist dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms

from dataset_tools import OwnDataset, make_train_test_val_split
#from plotting import show_img, loss_plots, accuracy_plots, plot_random_images, plot_image_comparison, plot_latent
from collections import defaultdict


from statsmodels.stats.contingency_tables import mcnemar
from models import LatentClassifier, VAE_CELL_CNN, CELL_CNN_CLASSIFIER, CELL_CNN_CLASSIFIER_2


cuda = torch.cuda.is_available()
print(cuda)

False


# Loading Data 

In [11]:
val = True
tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: x.view(-1)) # notice that we dont flatten when we are going to use CNN
])
dataset = OwnDataset(transform=tf)
# dataset = OwnDataset(transform=tf)
train_subset, test_subset, val_subset = make_train_test_val_split(dataset)


train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=True)

print(len(dataset), len(train_loader), len(test_loader), len(val_loader))



7774 78 25 20


# Loading Models 

In [24]:

from models import LatentClassifier_2, VAE_CELL_CNN, CELL_CNN_CLASSIFIER, CELL_CNN_CLASSIFIER_2
from loss_functions import loss_function
# import mse loss


# now we load the saved models into the model class
MODEL_DIR = "trained_models/"


latent_dim_256 = LatentClassifier_2(hidden_dim=512, latent_dim=256, num_classes=13)
latent_dim_256.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_256.__class__.__name__}_latent{latent_dim_256.latent_dim}_mean_best_model.pth",map_location=torch.device('cpu')))

latent_dim_2 = LatentClassifier_2(hidden_dim=4, latent_dim=2, num_classes=13)
latent_dim_2.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_2.__class__.__name__}_latent{latent_dim_2.latent_dim}_mean_best_model.pth",map_location=torch.device('cpu')))

Cell_CNN_Autoencoder = CELL_CNN_CLASSIFIER(input_dim=(3,68,68),hidden_dim=128, num_classes=13)
Cell_CNN_Autoencoder.load_state_dict(torch.load(MODEL_DIR + f"{Cell_CNN_Autoencoder.__class__.__name__}_best_model.pth",map_location=torch.device('cpu')))

# models = [vae_cnn_own]
models = [latent_dim_256, latent_dim_2, Cell_CNN_Autoencoder]
# models = [ae_cnn_own]



In [ ]:
latent_dim_256(image)

In [28]:
for image, moa in train_loader:
    print(latent_dim_256(image))
    print(moa)
    print("_________")
    
    
    
 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (13056x68 and 256x512)

In [ ]:
print(test_subset.dataset[2234])

(tensor([[[0.0745, 0.0667, 0.0745,  ..., 0.0824, 0.0784, 0.0784],
         [0.0706, 0.0627, 0.0745,  ..., 0.0784, 0.0706, 0.0784],
         [0.0745, 0.0745, 0.0745,  ..., 0.0784, 0.0745, 0.0745],
         ...,
         [0.0745, 0.0784, 0.0745,  ..., 0.0706, 0.0706, 0.0745],
         [0.0784, 0.0745, 0.0745,  ..., 0.0824, 0.0745, 0.0706],
         [0.0667, 0.0745, 0.0706,  ..., 0.0745, 0.0706, 0.0706]],

        [[0.3529, 0.3373, 0.3529,  ..., 0.1804, 0.2392, 0.3255],
         [0.3412, 0.3412, 0.3333,  ..., 0.1922, 0.2275, 0.2863],
         [0.2980, 0.3216, 0.3294,  ..., 0.2039, 0.2314, 0.2863],
         ...,
         [0.0902, 0.0784, 0.0824,  ..., 0.3686, 0.3569, 0.3529],
         [0.0902, 0.0863, 0.0824,  ..., 0.3804, 0.3686, 0.3647],
         [0.0824, 0.0824, 0.0863,  ..., 0.3922, 0.3725, 0.3725]],

        [[0.1569, 0.1686, 0.1608,  ..., 0.0902, 0.0980, 0.1137],
         [0.1608, 0.1529, 0.1412,  ..., 0.0824, 0.0902, 0.1059],
         [0.1255, 0.1373, 0.1373,  ..., 0.0824, 0.0902, 0

In [ ]:
test_subset.dataset[2234]

In [ ]:


# Load the predictions and true labels
model_A_predictions = np.load('trained_models/model_A_predictions.npy')
model_B_predictions = np.load('trained_models/model_B_predictions.npy')
true_labels = np.load('train_data/true_labels.npy')  # Assuming you have true labels


# making sure that the length is the same 
assert len(model_A_predictions) == len(model_B_predictions) == len(true_labels)

FileNotFoundError: [Errno 2] No such file or directory: 'trained_models/model_A_predictions.npy'

In [ ]:
# contingency table 
n00 = n01 = n10 = n11 = 0

for true, pred_A, pred_B in zip(true_labels, model_A_predictions, model_B_predictions):
    if pred_A == true and pred_B == true:
        n00 += 1
    elif pred_A == true and pred_B != true:
        n01 += 1
    elif pred_A != true and pred_B == true:
        n10 += 1
    else:
        n11 += 1

table = [[n00, n01], [n10, n11]]

table

In [ ]:
# Macnemar test + p-value 
result = mcnemar(table, exact=True)
print(f'statistic={result.statistic}, p-value={result.pvalue}')

alpha = 0.05
if result.pvalue < alpha:
    print('Significant difference in performance (reject H0)')
else:
    print('No significant difference in performance (fail to reject H0)')